#### Download data and save the log

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException, TimeoutException
import time
import logging

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a file handler for logging to a file
file_handler = logging.FileHandler('download_log.txt')
file_handler.setLevel(logging.INFO)

# Create a console handler for logging to the console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a logging format
formatter = logging.Formatter('%(asctime)s - %(message)s')
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)

# Add handlers to the logger
logger.addHandler(file_handler)
logger.addHandler(console_handler)

# Load the Excel file
excel_path = 'ChalinkAll.xlsx'
df = pd.read_excel(excel_path)

# Initialize the WebDriver (make sure the WebDriver is in your PATH)
driver = webdriver.Chrome()

# Iterate over the first 20 links in the Excel file
for index, row in df.iterrows():
    if index >= 2350:
        break
    link = row[0]  # Assuming links are in the first column (A)

    # Open the link in the browser
    driver.get(link)

    try:
        # Wait for the page to load
        wait = WebDriverWait(driver, 10)
        try:
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        except UnexpectedAlertPresentException:
            try:
                alert = driver.switch_to.alert
                alert_text = alert.text
                logger.warning(f"Unexpected alert present at {link} with text: {alert_text}")
                alert.accept()  # or alert.dismiss()
                continue  # Skip the rest of the loop and move to the next link
            except NoAlertPresentException:
                logger.warning(f"No alert present after detecting an UnexpectedAlertPresentException at {link}")

        # Wait for 3 seconds before proceeding
        time.sleep(3)

        try:
            # Find the element by XPath and click it
            download_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[7]/div/div/div/div[1]/div[2]/a'))
            )
            download_button.click()
    
            # Optionally, add a wait to ensure the file is downloaded before moving to the next link
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'body'))
            )
            logger.info(f'Successfully downloaded CSV from {link}')

        except UnexpectedAlertPresentException:
            # Handle unexpected alert
            try:
                alert = driver.switch_to.alert
                alert_text = alert.text
                logger.warning(f"Unexpected alert present at {link} with text: {alert_text}")
                alert.accept()  # or alert.dismiss()
            except NoAlertPresentException:
                logger.warning(f"No alert present after detecting an UnexpectedAlertPresentException at {link}")

    except TimeoutException as e:
        logger.error(f"Failed to download CSV from {link}: {e}")

# Close the browser
driver.quit()
